In [1]:
import json
import tiktoken
import torch
import instruction_dataset
import numpy as np
import time

import torch.nn as nn
import gpt_model
import train_model
import load_pretrained_weights
from gpt_download import download_and_load_gpt2
from functools import partial
settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")

from torch.utils.data import DataLoader

File already exists and is up-to-date: gpt2/124M/checkpoint
File already exists and is up-to-date: gpt2/124M/encoder.json
File already exists and is up-to-date: gpt2/124M/hparams.json
File already exists and is up-to-date: gpt2/124M/model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2/124M/model.ckpt.index
File already exists and is up-to-date: gpt2/124M/model.ckpt.meta
File already exists and is up-to-date: gpt2/124M/vocab.bpe
Output:
 Every effort moves you toward finding an ideal new way to practice something!

What makes us want to be on top of that?


File already exists and is up-to-date: gpt2/124M/checkpoint
File already exists and is up-to-date: gpt2/124M/encoder.json
File already exists and is up-to-date: gpt2/124M/hparams.json
File already exists and is up-to-date: gpt2/124M/model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2/124M/model.ckpt.index
File already exists and is up-to-date: gpt2/124M/model.ckpt.meta
File already exists 

In [2]:
# load instruction dataset
with open("instruction-data.json", "r") as file:
    data = json.load(file)

print("Number of entries: ", len(data))
print("Example entry:\n", data[50])


Number of entries:  1100
Example entry:
 {'instruction': 'Identify the correct spelling of the following word.', 'input': 'Ocassion', 'output': "The correct spelling is 'Occasion.'"}


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if torch.backends.mps.is_available:
#    device = torch.device("mps") # Apple Silicon
tokenizer = tiktoken.get_encoding("gpt2")

gpt = gpt_model.GPTModel(
    vocab_size = 50257,
    context_length = 1024,
    emb_dim = 768,
    num_heads = 12,
    num_layers = 12,
    drop_rate = 0.0,
    qkv_bias = True
)
gpt.eval()

GPTModel(
  (token_embedding): Embedding(50257, 768)
  (position_embedding): Embedding(1024, 768)
  (dropout_embedding): Dropout(p=0.0, inplace=False)
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (attention): Attention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerBlock(
      (attention): Attention(
        (

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = tiktoken.get_encoding("gpt2")

In [5]:
load_pretrained_weights.load_weights_into_gpt(gpt, params)
gpt.eval()

GPTModel(
  (token_embedding): Embedding(50257, 768)
  (position_embedding): Embedding(1024, 768)
  (dropout_embedding): Dropout(p=0.0, inplace=False)
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (attention): Attention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerBlock(
      (attention): Attention(
        (

In [6]:
dataset = instruction_dataset.InstructionDataset(data, tokenizer)
dataset.__getitem__(0)

[21106,
 318,
 281,
 12064,
 326,
 8477,
 257,
 4876,
 13,
 19430,
 257,
 2882,
 326,
 20431,
 32543,
 262,
 2581,
 13,
 220,
 198,
 198,
 21017,
 46486,
 25,
 198,
 36,
 2100,
 4985,
 262,
 1708,
 9546,
 416,
 25449,
 340,
 656,
 262,
 24993,
 1813,
 13,
 198,
 198,
 21017,
 23412,
 25,
 198,
 19503,
 521,
 14610,
 1545,
 198,
 198,
 21017,
 18261,
 25,
 198,
 464,
 24993,
 286,
 262,
 1813,
 9546,
 366,
 19503,
 521,
 1,
 318,
 11491,
 11,
 262,
 3376,
 24993,
 318,
 366,
 6726,
 1911]

In [7]:
test_input = dataset.format_input(data[50])
response = f"\n\n### Response:\n{data[50]['output']}"
print(test_input + response)

Below is an instruction that describes a task. Write a response that appropriately completes the request. 

### Instruction:
Identify the correct spelling of the following word.

### Input:
Ocassion

### Response:
The correct spelling is 'Occasion.'


In [8]:
test_input = dataset.format_input(data[999])
response = f"\n\n### Response:\n{data[999]['output']}"
print(test_input + response)

Below is an instruction that describes a task. Write a response that appropriately completes the request. 

### Instruction:
What is an antonym of 'complicated'?

### Response:
An antonym of 'complicated' is 'simple'.


In [9]:
# split data into training, test, and validation sets
train_portion = int(len(data) * 0.85)
test_portion = int(len(data) * 0.1)
val_portion = len(data) - train_portion - test_portion

train_data = data[:train_portion]
test_data = data[train_portion:train_portion + test_portion]
val_data = data[train_portion + test_portion:]

print("Training set length: ", len(train_data))
print("Test set length: ", len(test_data))
print("Validation set length: ", len(val_data))

Training set length:  935
Test set length:  110
Validation set length:  55


In [10]:
inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]
batch = (inputs_1, inputs_2, inputs_3)

inputs, targets = instruction_dataset.collate(batch)
print(inputs)
print()
print(targets)

tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]])

tensor([[    1,     2,     3,     4, 50256],
        [    6, 50256,  -100,  -100,  -100],
        [    8,     9, 50256,  -100,  -100]])


In [11]:
num_workers = 0
batch_size = 8
torch.manual_seed(123)

collate_function = partial(
    instruction_dataset.collate,
    device=device,
    allowed_max_length=1024
)

train_dataset = instruction_dataset.InstructionDataset(train_data, tokenizer)
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    collate_fn = collate_function,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True
)

val_dataset = instruction_dataset.InstructionDataset(val_data, tokenizer)
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    collate_fn=collate_function,
    shuffle=False,
    num_workers=num_workers,
    drop_last=False
)

test_dataset = instruction_dataset.InstructionDataset(test_data, tokenizer)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    collate_fn=collate_function,
    shuffle=False,
    num_workers=num_workers,
    drop_last=False
)

print("Train loader")
for input_batch, target_batch in train_loader:
    print(input_batch.shape, target_batch.shape)

Train loader
torch.Size([8, 62]) torch.Size([8, 62])
torch.Size([8, 77]) torch.Size([8, 77])
torch.Size([8, 74]) torch.Size([8, 74])
torch.Size([8, 69]) torch.Size([8, 69])
torch.Size([8, 66]) torch.Size([8, 66])
torch.Size([8, 73]) torch.Size([8, 73])
torch.Size([8, 81]) torch.Size([8, 81])
torch.Size([8, 68]) torch.Size([8, 68])
torch.Size([8, 63]) torch.Size([8, 63])
torch.Size([8, 76]) torch.Size([8, 76])
torch.Size([8, 63]) torch.Size([8, 63])
torch.Size([8, 69]) torch.Size([8, 69])
torch.Size([8, 68]) torch.Size([8, 68])
torch.Size([8, 78]) torch.Size([8, 78])
torch.Size([8, 70]) torch.Size([8, 70])
torch.Size([8, 80]) torch.Size([8, 80])
torch.Size([8, 72]) torch.Size([8, 72])
torch.Size([8, 67]) torch.Size([8, 67])
torch.Size([8, 84]) torch.Size([8, 84])
torch.Size([8, 69]) torch.Size([8, 69])
torch.Size([8, 81]) torch.Size([8, 81])
torch.Size([8, 72]) torch.Size([8, 72])
torch.Size([8, 70]) torch.Size([8, 70])
torch.Size([8, 66]) torch.Size([8, 66])
torch.Size([8, 69]) torch.S

In [12]:
# calculate initial loss for training and validation sets
gpt.to(device)
torch.manual_seed(123)

with torch.no_grad():
    train_loss = train_model.calc_loss_loader(data_loader=train_loader, model=gpt, device=device, num_batches=5)
    val_loss = train_model.calc_loss_loader(data_loader=val_loader, model=gpt, device=device, num_batches=5)

print("Training loss: ", train_loss)
print("Validation loss: ", val_loss)

Training loss:  4.285567283630371
Validation loss:  4.165820121765137


In [14]:
# fine-tune LLM 

#TODO remove from class in instruction_dataset.py
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request. "
        f"\n\n### Instruction:\n{entry['instruction']}"
    )
    
    input_text = (f"\n\n### Input:\n{entry['input']}" if entry['input'] else "")
    
    return instruction_text + input_text

start_time = time.time()
torch.manual_seed(123)
optimizer = torch.optim.AdamW(gpt.parameters(), lr=0.00005, weight_decay=0.1)
num_epochs = 2

train_losses, val_losses, tokens_seen = train_model.train_model_simple(
    model=gpt,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    device=device,
    num_epochs=num_epochs,
    eval_freq=5,
    eval_iter=5,
    start_context=format_input(val_data[0]),
    tokenizer=tokenizer
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes")

Epoch 1, Step 000000 : Train loss 3.209,Validation loss 3.233
Below is an instruction that describes a task. Write a response that appropriately completes the request.   ### Instruction: Convert the active sentence to passive: 'The chef cooks the meal every day.'  ### Instruction: Convert the active sentence to passive: 'The chef cooks the meal every day.'  ### Instruction: Convert the active sentence to passive: 'The chef cooks the meal every day.'  ### Instruction
Below is an instruction that describes a task. Write a response that appropriately completes the request.   ### Instruction: Convert the active sentence to passive: 'The chef cooks the meal every day.'  ### Response: Write a response that appropriately completes the request.  ### Response: Write a response that appropriately completes the request.  ### Response: Write a response that appropriately completes the request.  ### Response:
Below is an instruction that describes a task. Write a response that appropriately complet

In [16]:
torch.manual_seed(123)

for entry in test_data[:3]: # first 3 test set samples
    input_text = format_input(entry)
    token_ids = train_model.generate(
        model=gpt,
        index=train_model.text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=256,
        context_size=1024,
        eos_id=50256
    )
    
    generated_text = train_model.token_ids_to_text(token_ids, tokenizer)
    
    response_text = (
        generated_text[len(input_text):]
        .replace("### Response:", "")
        .strip()
    )
    
    print(input_text)
    print(f"\nCorrect response:\n>> {entry['output']}")
    print(f"\nModel reponse:\n>> {response_text.strip()}")

Below is an instruction that describes a task. Write a response that appropriately completes the request. 

### Instruction:
Rewrite the sentence using a simile.

### Input:
The car is very fast.

Correct response:
>> The car is as fast as lightning.

Model reponse:
>> The car is as fast as a horse.
Below is an instruction that describes a task. Write a response that appropriately completes the request. 

### Instruction:
What type of cloud is typically associated with thunderstorms?

Correct response:
>> The type of cloud typically associated with thunderstorms is cumulonimbus.

Model reponse:
>> A type of thunderstorm is a tropical storm.
Below is an instruction that describes a task. Write a response that appropriately completes the request. 

### Instruction:
Name the author of 'Pride and Prejudice'.

Correct response:
>> Jane Austen.

Model reponse:
>> The author of 'Pride and Prejudice' is Robert Frost.


In [17]:
from tqdm import tqdm

for i, entry in tqdm(enumerate(test_data), total=len(test_data)):
    input_text = format_input(entry)
    token_ids = train_model.generate(
        model=gpt,
        index=train_model.text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=256,
        context_size=1024,
        eos_id=50256
    )
    
    generated_text = train_model.token_ids_to_text(token_ids, tokenizer)
    
    response_text = (
        generated_text[len(input_text):]
        .replace("### Response:", "")
        .strip()
    )
    test_data[i]["model_response"] = response_text
    
    with open("intruction-data-with-response.json", "w") as file:
        json.dump(test_data, file, indent=4)

100%|██████████| 110/110 [01:24<00:00,  1.30it/s]


In [18]:
print(test_data[10])

{'instruction': 'What is the capital of Denmark?', 'input': '', 'output': 'The capital of Denmark is Copenhagen.', 'model_response': 'The capital of Denmark is Copenhagen.'}


In [21]:
def print_model_response(index):
    print(f"Instruction: {test_data[index]['instruction']}")
    print()
    print(f"Input: {test_data[index]['input']}")
    print()
    print(f"Output: {test_data[index]['output']}")
    print()
    print(f"Model response: {test_data[index]['model_response']}")
    print()

In [22]:
print_model_response(20)

Instruction: Pick out the adjective from the following list.

Input: run, tall, quickly

Output: The correct adjective from the list is 'tall.'

Model response: The adjective from the list is 'run'.



In [24]:
print_model_response(100)

Instruction: Generate a sentence using the word 'generous'.

Input: 

Output: He is very generous and always helps those in need.

Model response: He was very generous and always made sure that everyone was given a present.



In [25]:
print_model_response(75)

Instruction: Translate 'Can I have some water?' into French.

Input: 

Output: Puis-je avoir de l'eau?

Model response: The French translation of 'Can I have some water?' is 'C'est la vie'?



In [26]:
print_model_response(23)

Instruction: Categorize the given list of animals.

Input: Shark, Dolphin, Trout

Output: Fish: Shark, Trout
Mammals: Dolphin

Model response: Dolphins, Shark, Dolphin



In [27]:
print_model_response(30)

Instruction: What is the chemical formula for magnesium sulfate?

Input: 

Output: The chemical formula for magnesium sulfate is MgSO4.

Model response: The chemical formula for magnesium sulfate is MgS.S.M.H.S.



In [28]:
print_model_response(50)

Instruction: Edit the given text to ensure all plural nouns are spelled correctly.

Input: The birds sings beautiful songs.

Output: The birds sing beautiful songs.

Model response: The birds sings beautiful songs.



In [29]:
print_model_response(60)

Instruction: What is an antonym of 'old'?

Input: 

Output: young.

Model response: An antonym of 'old' is 'young'.

